# FICO challenge dataset
## 1: Clean Data and and build FFNN

In [1]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.sparse

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras.backend as K

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

Using TensorFlow backend.


In [2]:
import os
RESULT_PATH = './results'

if not os.path.exists(RESULT_PATH):
        os.mkdir(RESULT_PATH)

In [3]:
import pandas as pd
data = pd.read_csv("Data/heloc_dataset_v1.csv")
print(data.shape)
data.head()

(10459, 24)


,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80


In [4]:
# import pandas_profiling
# profile = pandas_profiling.ProfileReport(data, minimal=False)
# profile

In [5]:
dups = data.duplicated(subset=None, keep='first')
data[dups]['RiskPerformance'].value_counts()

Bad     323
Good    264
Name: RiskPerformance, dtype: int64

Of the 588 duplicates (all -9 values, i.e. not found at bureaux) there's a 55% bad rate

In [6]:
# Save target y and dummy the X matrix

CLASS = 'RiskPerformance'

# Split X and y
X = data.drop(columns=[CLASS])
y = data[CLASS]
class_names = sorted(y.unique(),  reverse=True)

y.value_counts()

Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64

Encode y

In [7]:
import numpy as np
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(y)
# y_onehot = encoder.transform(y)
# y_onehot = 1-y_onehot

y_onehot = pd.get_dummies(y)[['Bad']]
np.array(np.unique(y_onehot, return_counts=True)).T

array([[   0, 5000],
       [   1, 5459]], dtype=int64)

### Scale X

In [8]:
# Scale the Training Data Matrix

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)

# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()

## Standard Scaler is best performin
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(data.values)
# X = scaler.transform(data.values)

## No scaling
# X = data.values



### Alternatively, Bin X following the Chen, lin, Rudin segmentations

In [9]:
input_names = list(X.columns)
input_names_bin = [x + '_bin' for x in input_names]

X1 = pd.DataFrame()

# ExternalRiskEstimate
X1[input_names_bin[0]]=pd.cut(x=X[input_names[0]], 
                                      bins=[-float("inf"),-9,-8,-7,0,63,70,75,80,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[0]].value_counts(sort=True)

# TradeOpenTime
X1[input_names_bin[1]]=pd.cut(x=X[input_names[1]], 
                                      bins=[-float("inf"),-9,-8,-7,0,91,134,263,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[1]].value_counts(sort=True)


X1[input_names_bin[2]]=pd.cut(x=X[input_names[2]], 
                                      bins=[-float("inf"),-9,-8,-7,0,19,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[2]].value_counts(sort=True)

X1[input_names_bin[3]]=pd.cut(x=X[input_names[3]], 
                                      bins=[-float("inf"),-9,-8,-7,0,48,69,96,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[3]].value_counts(sort=True)

# NumSatisfactoryTrades
X1[input_names_bin[4]]=pd.cut(x=X[input_names[4]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,5,12,21,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[4]].value_counts(sort=True)

# TradeFrequency
X1[input_names_bin[5]]=pd.cut(x=X[input_names[5]], 
                                      bins=[-float("inf"),-9,-8,-7,0,1,2,11,12,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[5]].value_counts(sort=True)

X1[input_names_bin[6]]=pd.cut(x=X[input_names[6]], 
                                      bins=[-float("inf"),-9,-8,-7,0,1,7,9,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[6]].value_counts(sort=True)

X1[input_names_bin[11]]=pd.cut(x=X[input_names[11]], 
                                      bins=[-float("inf"),-9,-8,-7,0,9,16,27,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[11]].value_counts(sort=True)


X1[input_names_bin[12]]=pd.cut(x=X[input_names[12]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,3,6,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[12]].value_counts(sort=True)

# Delinquency
X1[input_names_bin[7]]=pd.cut(x=X[input_names[7]], 
                                      bins=[-float("inf"),-9,-8,-7,0,59,84,89,96,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[7]].value_counts(sort=True)

X1[input_names_bin[8]]=pd.cut(x=X[input_names[8]], 
                                      bins=[-float("inf"),-9,-8,-7,0,17,32,47,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[8]].value_counts(sort=True)

X1[input_names_bin[9]]=pd.cut(x=X[input_names[9]], 
                                      bins=[-float("inf"),-9,-8,-7,0,5,6,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[9]].value_counts(sort=True)

X1[input_names_bin[10]]=pd.cut(x=X[input_names[10]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[10]].value_counts(sort=True)

# Installment
X1[input_names_bin[13]]=pd.cut(x=X[input_names[13]], 
                                      bins=[-float("inf"),-9,-8,-7,0,35,46,57,84,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[13]].value_counts(sort=True)

X1[input_names_bin[18]]=pd.cut(x=X[input_names[18]], 
                                      bins=[-float("inf"),-9,-8,-7,0,35,70,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[18]].value_counts(sort=True)

X1[input_names_bin[20]]=pd.cut(x=X[input_names[20]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,3,11,13,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[20]].value_counts(sort=True)


#Inquiry
X1[input_names_bin[14]]=pd.cut(x=X[input_names[14]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,8,22,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[14]].value_counts(sort=True)

X1[input_names_bin[15]]=pd.cut(x=X[input_names[15]], 
                                      bins=[-float("inf"),-9,-8,-7,0,1,4,8,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[15]].value_counts(sort=True)

X1[input_names_bin[16]]=pd.cut(x=X[input_names[16]], 
                                      bins=[-float("inf"),-9,-8,-7,0,2,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[16]].value_counts(sort=True)

# Revolving balance
X1[input_names_bin[17]]=pd.cut(x=X[input_names[17]], 
                                      bins=[-float("inf"),-9,-8,-7,0,14,37,72,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[17]].value_counts(sort=True)

X1[input_names_bin[19]]=pd.cut(x=X[input_names[19]], 
                                      bins=[-float("inf"),-9,-8,-7,0,3,4,7,11,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[19]].value_counts(sort=True)

# Utilisation
X1[input_names_bin[21]]=pd.cut(x=X[input_names[21]], 
                                      bins=[-float("inf"),-9,-8,-7,0,1,2,3,5,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[21]].value_counts(sort=True)

# Trade with balance
X1[input_names_bin[22]]=pd.cut(x=X[input_names[22]], 
                                      bins=[-float("inf"),-9,-8,-7,0,47,66,73,86,float("inf")]
#                                       ,labels=["1.799", "1.017", "0.269","-0.326"]
                                     )
X1[input_names_bin[22]].value_counts(sort=True)

X1.head()

,ExternalRiskEstimate_bin,MSinceOldestTradeOpen_bin,MSinceMostRecentTradeOpen_bin,AverageMInFile_bin,NumSatisfactoryTrades_bin,NumTrades60Ever2DerogPubRec_bin,NumTrades90Ever2DerogPubRec_bin,NumTotalTrades_bin,NumTradesOpeninLast12M_bin,PercentTradesNeverDelq_bin,...,PercentInstallTrades_bin,NetFractionInstallBurden_bin,NumInstallTradesWBalance_bin,MSinceMostRecentInqexcl7days_bin,NumInqLast6M_bin,NumInqLast6Mexcl7days_bin,NetFractionRevolvingBurden_bin,NumRevolvingTradesWBalance_bin,NumBank2NatlTradesWHighUtilization_bin,PercentTradesWBalance_bin
0,"(0.0, 63.0]","(134.0, 263.0]","(0.0, 19.0]","(69.0, 96.0]","(12.0, 21.0]","(2.0, 11.0]","(-7.0, 0.0]","(16.0, 27.0]","(0.0, 2.0]","(59.0, 84.0]",...,"(35.0, 46.0]","(-9.0, -8.0]","(0.0, 2.0]","(-7.0, 0.0]","(-7.0, 0.0]","(-7.0, 0.0]","(14.0, 37.0]","(7.0, 11.0]","(0.0, 1.0]","(66.0, 73.0]"
1,"(0.0, 63.0]","(0.0, 91.0]","(0.0, 19.0]","(0.0, 48.0]","(0.0, 2.0]","(2.0, 11.0]","(1.0, 7.0]","(0.0, 9.0]","(-7.0, 0.0]","(96.0, inf]",...,"(57.0, 84.0]","(-9.0, -8.0]","(-9.0, -8.0]","(-7.0, 0.0]","(-7.0, 0.0]","(-7.0, 0.0]","(-7.0, 0.0]","(-7.0, 0.0]","(-9.0, -8.0]","(-7.0, 0.0]"
2,"(63.0, 70.0]","(0.0, 91.0]","(0.0, 19.0]","(0.0, 48.0]","(5.0, 12.0]","(-7.0, 0.0]","(-7.0, 0.0]","(0.0, 9.0]","(3.0, 6.0]","(96.0, inf]",...,"(35.0, 46.0]","(35.0, 70.0]","(0.0, 2.0]","(-7.0, 0.0]","(1.0, 4.0]","(2.0, inf]","(37.0, 72.0]","(3.0, 4.0]","(0.0, 1.0]","(73.0, 86.0]"
3,"(63.0, 70.0]","(134.0, 263.0]","(0.0, 19.0]","(69.0, 96.0]","(21.0, inf]","(0.0, 1.0]","(0.0, 1.0]","(27.0, inf]","(2.0, 3.0]","(89.0, 96.0]",...,"(46.0, 57.0]","(70.0, inf]","(3.0, 11.0]","(-7.0, 0.0]","(4.0, 8.0]","(2.0, inf]","(37.0, 72.0]","(4.0, 7.0]","(2.0, 3.0]","(86.0, inf]"
4,"(80.0, inf]","(263.0, inf]","(19.0, inf]","(96.0, inf]","(5.0, 12.0]","(-7.0, 0.0]","(-7.0, 0.0]","(9.0, 16.0]","(-7.0, 0.0]","(96.0, inf]",...,"(0.0, 35.0]","(70.0, inf]","(0.0, 2.0]","(-7.0, 0.0]","(0.0, 1.0]","(0.0, 2.0]","(37.0, 72.0]","(0.0, 3.0]","(-7.0, 0.0]","(73.0, 86.0]"


One hot encode

In [10]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\brizio\Documents\PythonNB\FICOchallenge


In [11]:
os.chdir(cwd)
save_d = False
if save_d == True:
    # Save modified data (to save column names etc.)
    X1.to_csv("./Data/encoded_data.csv", index=False)
    y.to_csv("./Data/y_data.csv", index=False)

In [13]:
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np

# One-hot
# data_onehot = pd.get_dummies(data)
X_onehot = pd.get_dummies(X1)
def drop_cardinality_1(df):
        res = df
        for col in df.columns:
                if len(df[col].unique()) == 1:
                        res = res.drop(col,axis=1)
        return res
X_onehot = drop_cardinality_1(X_onehot)

y_onehot = pd.get_dummies(y)[['Bad']]

print(np.array(np.unique(y_onehot, return_counts=True)).T)

X_train, X_test, y_train, y_test, X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X1, y, X_onehot, y_onehot, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)

[[   0 5000]
 [   1 5459]]
(7844, 145) (2615, 145) (7844, 1) (2615, 1)


In [14]:
# X_onehot_train = scipy.sparse.csc_matrix(X_onehot_train.to_numpy())
# X_onehot_test = scipy.sparse.csc_matrix(X_onehot_test.to_numpy())